In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Loguru — Logging made easy

https://github.com/Delgan/loguru

In [4]:
from loguru import logger

In [5]:
logger.debug("Hello, cool debugger")

2019-04-23 16:36:06.397 | DEBUG    | __main__:<module>:1 - Hello, cool debugger


# more-itertool

In [4]:
from more_itertools import peekable, chunked

In [3]:
 p = peekable(['a', 'b'])
 p.peek()
 next(p)

'a'

'a'

In [5]:
list(chunked([1, 2, 3, 4, 5, 6], 3))

[[1, 2, 3], [4, 5, 6]]

# MonkeyType
MonkeyType collects runtime types of function arguments and return values, and can automatically generate stub files or even add draft type annotations directly to your Python code based on the types collected at runtime.

# requests-async
Brings support for async/await syntax to Python's fabulous requests library.

# pyright
Static type checker for Python


# black
The uncompromising Python code formatter https://black.readthedocs.io/en/stable/

# Custom Magic CMD and C++ Kernel

In [6]:
from IPython.core.magic import register_cell_magic

In [12]:
@register_cell_magic
def cpp(line, cell):
    """Compile, execute C++ code, and return stdout"""
    # First retrieve the current IPython interpreter instance
    ip = get_ipython()
    
    # Define source and executable filenames
    source_filename = '_temp.cpp'
    program_filename = '_temp'
    
    # write code to C++ file
    with open(source_filename, 'w') as f:
        f.write(cell)
    
    # compile C++ code into an executable
    compile = ip.getoutput('g++ {0:s} -o {1:s}'.format(source_filename, program_filename))
    
    # execute the exectuable and return output
    output = ip.getoutput('./{0:s}'.format(program_filename))
    
    print('\n'.join(output))    

In [13]:
%%cpp
#include <iostream>
int main()
{
    std::cout << "Hello World";
}

Hello World


> Cell magic only exist in runtime
    - pack into extension

```python
# save above cpp() as cpp_ext.py appending below
def load_ipython_extension(ipython):
    ipython.register_magic_function(cpp, 'cell')
```
    - then load with `%load_ext cpp_ext`

### C++ Kernel

In [27]:
%mkdir cpp

In [31]:
%%writefile cpp/cpp_kernel.py

import os
import os.path as op
import tempfile

# import `getoutput()` func by IPython
# allowing sys call from Python
from IPython.utils.process import getoutput

def exec_cpp(code):
    """Compile-execute C++ code and return stdout"""
    # create a temp dir to be deleted end of 'with' context
    with tempfile.TemporaryDirectory() as tmpdir:
        # define source and exec filenames
        source_path = op.join(tmpdir, 'temp.cpp')
        program_path = op.join(tmpdir, 'temp')
        # write code to C++ file
        with open(source_path, 'w') as f:
            f.write(code)
        # compile into exec
        os.system("g++ {0:s} -o {1:s}".format(
            source_path, program_path))
        # execute program and return output
        return getoutput(program_path)

Writing cpp/cpp_kernel.py


In [32]:
%%writefile -a cpp/cpp_kernel.py

"""C++ wrapper kernel."""
from ipykernel.kernelbase import Kernel

class CppKernel(Kernel):
    "Kernel Info"
    implementation = 'C++'
    implementation_version = '1.0'
    language = 'C++'
    language_version = '1.0'
    language_info = {'name': 'c++',
                     'mimetype': 'text/plain'}
    banner = "C++ kernel"
    
    def do_execute(self, code, silent,
                  store_history=True,
                  user_expressions=None,
                  allow_stdin=False):
        """This func called as code cell executed"""
        if not silent:
            # run C++ code and get output
            output = exec_cpp(code)
            # send back result to frontned
            stream_content = {'name': 'stdout',
                             'text': output}
            self.send_response(self.iopub_socket,
                              'stream', stream_content)
            return {'status': 'ok',
                   # base class increments the execution count
                   'execution_count': self.execution_count,
                   'payload': [],
                   'user_expressions': {},
                   }

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(kernel_class=CppKernel)

Appending to cpp/cpp_kernel.py


In [33]:
%%writefile cpp/kernel.json

{
    "argv": ["python",
            "cpp/cpp_kernel.py",
            "-f",
            "{connection_file}"
            ],
    "display_name": "C++"
}

Writing cpp/kernel.json


In [34]:
!jupyter kernelspec install --replace --user cpp

[InstallKernelSpec] Installed kernelspec cpp in /Users/Ocean/Library/Jupyter/kernels/cpp


In [35]:
!jupyter kernelspec list

Available kernels:
  cpp          /Users/Ocean/Library/Jupyter/kernels/cpp
  ir           /Users/Ocean/Library/Jupyter/kernels/ir
  julia-0.6    /Users/Ocean/Library/Jupyter/kernels/julia-0.6
  python3      /Users/Ocean/.virtualenvs/general/bin/../share/jupyter/kernels/python3
